In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
device

device(type='cuda')

In [5]:
cache_dir = "model_cache"

In [7]:
tokenizer_lm = AutoTokenizer.from_pretrained("meta-math/MetaMath-7B-V1.0", cache_dir=cache_dir, padding_side='right')
model_lm = AutoModelForCausalLM.from_pretrained("meta-math/MetaMath-7B-V1.0", cache_dir=cache_dir)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
text = "John has 2 balls and Ann took 1 from him. How many balls does John has now?"

In [9]:
token_ids = tokenizer_lm(text,max_length=2048,padding="max_length", truncation=True, return_tensors='pt').to(device)

In [12]:
token_ids["input_ids"].shape

torch.Size([1, 2048])

In [15]:
encoded_input = tokenizer_lm(
    text,
    max_length=1024,   # Set the fixed length
    padding='max_length', # Pad to max length
    truncation=True,    # Truncate if longer than max length
    return_tensors='pt' # Return as PyTorch tensors
)

# Move input IDs to the appropriate device
input_ids = encoded_input['input_ids']
attention_mask = encoded_input['attention_mask']

In [16]:
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model_lm.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=500,              # Maximum number of new tokens to generate
        pad_token_id=tokenizer_lm.pad_token_id,  # Padding token ID
        eos_token_id=tokenizer_lm.eos_token_id,  # End-of-sequence token ID
        no_repeat_ngram_size=2,
    )

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


KeyboardInterrupt: 